In [1]:
import glob
from pathlib import Path
import json

from pandas_profiling import ProfileReport
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import cv2 as cv

In [2]:
df = pd.read_csv('/home/rchaban/session_test.csv', index_col=None)
print(df['ses_status'].value_counts())
# df = df[df['ses_status'] == 'success']

success    53925
fail          75
Name: ses_status, dtype: int64


/opt/anaconda3/envs/pharma/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (315,316) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [ ]:
df_failed = df[df['ses_status'] == 'fail']

In [ ]:
import shutil
for file in df_failed.fi_filename.tolist():
    shutil.copy(f'/data/500gb/pharmapack/Enrollment/cropped/{file}.png', f'/data/500gb/pharmapack/Enrollment/not_processed/{file}.png')

In [ ]:
df_failed.ses_trcbk.value_counts()

In [ ]:
df_failed.fi_filename.tolist()

In [ ]:
#df_min_IOU = df.groupby('fi_package_class').median().sort_values(by=['er_MIN_IOU']).reset_index()[['fi_package_class', 'er_MIN_IOU']]
sns.displot(x=df.groupby('fi_package_class').median().sort_values(by=['er_MIN_IOU']).reset_index().er_MIN_IOU.to_numpy(), 
            binwidth=0.05, height=10, facet_kws=dict(margin_titles=True))

In [ ]:
# sns.displot(data=df_min_IOU, x='fi_package_class', y='er_MIN_IOU')
sns.displot(x=df_min_IOU.er_MIN_IOU.to_numpy(), binwidth=0.05, height=10, facet_kws=dict(margin_titles=True))

In [ ]:
algorithm = 'MSER'
mode = 'TXT'
metric = 'PRC'
arr = df[f'er_{algorithm}_{mode}_{metric}'].to_numpy()
arr = arr[arr < 1.0]
sns.displot(x=arr, binwidth=0.05, height=10, facet_kws=dict(margin_titles=True))

In [ ]:
df_copy = df.copy(deep=True)
cols = df.columns
for col in cols:
    if col.find('ALL') != -1:
        df_copy[col.replace('ALL', 'DIFF')] = df_copy[col] - df_copy[col.replace('ALL', 'TXT')]
        
df_copy

In [ ]:
df_mean = df_copy.groupby('fi_package_class').mean().reset_index()
df_median = df_copy.groupby('fi_package_class').median().reset_index()
df_std = df_copy.groupby('fi_package_class').std().reset_index()
df_var = df_copy.groupby('fi_package_class').var().reset_index()

In [ ]:
sns.displot(x=df_var['er_MSER_DIFF_IOU'], binwidth=0.01, height=10, facet_kws=dict(margin_titles=True))

In [3]:
df_SVM = df[df['ses_status'] == 'success']
# df_SVM = df_SVM.sample(frac=0.5)
df_SVM = df.drop(columns=df.columns[df.isna().all()].tolist())
df_CLASS = df_SVM['fi_package_class']
# df_SVM = df_SVM.rename(columns={'fi_package_class': 'er_package_class'})
df_SVM = df_SVM[df_SVM.columns[pd.Series(df_SVM.columns).str.startswith('er')]]
df_SVM = df_SVM.fillna(df.mean())

In [4]:
df_SVM[df_SVM.isna().any(axis=1)]

,er_LS_ALL_ACC,er_LS_ALL_FN,er_LS_ALL_FP,er_LS_ALL_IOU,er_LS_ALL_PRC,er_LS_ALL_R,er_LS_ALL_SNS,er_LS_ALL_SPC,er_LS_ALL_TN,er_LS_ALL_TP,...,er_MSER_ALL_TP,er_MSER_TXT_ACC,er_MSER_TXT_FN,er_MSER_TXT_FP,er_MSER_TXT_IOU,er_MSER_TXT_PRC,er_MSER_TXT_SNS,er_MSER_TXT_SPC,er_MSER_TXT_TN,er_MSER_TXT_TP


In [5]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn import metrics

In [6]:
X = df_SVM.to_numpy()
y = df_CLASS.to_numpy()

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [8]:
y_train.shape

(48600,)

In [ ]:
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(X_train, y_train)

In [ ]:
clf.score(X_test, y_test)

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
np.mean(y_pred == y_test)

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred, pos_label=2)

In [ ]:
probs = clf.predict_proba(X_test)

In [ ]:
clf_prob = make_pipeline(StandardScaler(), SVC(gamma='auto', probability=True))
clf_prob.fit(X_train, y_train)

In [ ]:
probs = clf_prob.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

In [ ]:
from itertools import cycle
colors = ['aqua', 'darkorange', 'cornflowerblue']
i = 1
plt.plot(fpr[i], tpr[i], color=colors[0], lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.show()


In [ ]:
len(fpr)